# Chapter05

In [2]:
from dotenv import load_dotenv

loaded = load_dotenv()

## 5.1 RunnableLambda

`RunnableLambda` allows a function to be converted into a `Runnable` type, enabling it to be added to a chain.

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
    ]
)

model = ChatOpenAI(model="gpt-4o", temperature=0)

output_parser = StrOutputParser()

In [4]:
from langchain_core.runnables import RunnableLambda

def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | output_parser | RunnableLambda(upper)

output = chain.invoke({"input": "Hello!"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


We can also use `decorator` to obtain the same effect.

In [6]:
from langchain_core.runnables import chain

@chain
def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | output_parser | upper

output = chain.invoke({"input": "Hello!"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


**Automatic conversion** is the most commonly used approach. Whenever nearby objects in the chain are a `Runnable`, the object is automatically converted into a `Runnable`. 

In [6]:
def upper(text: str) -> str:
    return text.upper()

chain = prompt | model | output_parser | upper

output = chain.invoke({"input": "Hello!"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


### streaming mode

Because `upper` batches all its input and returns a single result, calling the chain with .stream() won't emit anything until it's fully done.

In [8]:
from typing import Iterator

def upper(input_stream: Iterator[str]) -> Iterator[str]:
    for text in input_stream:
        yield text.upper()

chain = prompt | model | StrOutputParser() | upper

for chunk in chain.stream({"input": "Hello! Say something in 50 words."}):
    print(chunk, end="", flush=True)

HELLO! IT'S GREAT TO CONNECT WITH YOU. I HOPE YOU'RE HAVING A WONDERFUL DAY. IF THERE'S ANYTHING SPECIFIC YOU'D LIKE TO DISCUSS OR ANY QUESTIONS YOU HAVE, FEEL FREE TO ASK. I'M HERE TO HELP WITH INFORMATION, ADVICE, OR JUST A FRIENDLY CHAT. LOOKING FORWARD TO OUR CONVERSATION!

## 5.2 RunnableParallel

Define basic conponents

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)
output_parser = StrOutputParser()

Create optimistic chain

In [28]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見をください。"),
        ("human", "{topic}"),
    ]
)
optimistic_chain = optimistic_prompt | model | output_parser

Create pessimistic chain

In [31]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください。"),
        ("human", "{topic}"),
    ]
)
pessimistic_chain = pessimistic_prompt | model | output_parser

Let model output optimistic answer and pessimistic answer parallelly

In [32]:
import pprint
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

output = parallel_chain.invoke({"topic": "生成AIの進化について"})
pprint.pprint(output)

{'optimistic_opinion': '生成AIの進化は本当にワクワクするものですね！この技術の進歩は、私たちの生活をより便利で豊かにしてくれる可能性を秘めています。例えば、クリエイティブなプロジェクトの支援や、日常業務の効率化、さらには新しいアイデアの発掘など、さまざまな分野での活用が期待されています。\n'
                       '\n'
                       'また、生成AIは教育や医療、エンターテインメントなど、多くの分野で革新をもたらすことができるでしょう。これからの進化によって、私たちがまだ想像もしていないような新しい可能性が開かれるかもしれません。技術が進化することで、より多くの人々がその恩恵を受けられるようになり、世界がより良い場所になることを期待しています！',
 'pessimistic_opinion': '生成AIの進化は確かに目覚ましいものがありますが、悲観的に考えると、これにはいくつかの懸念が伴います。まず、生成AIが進化することで、人間の仕事が奪われる可能性が高まります。特にクリエイティブな分野での自動化が進むと、多くの人が職を失うかもしれません。\n'
                        '\n'
                        'また、生成AIが進化することで、フェイクニュースや偽情報の拡散がさらに容易になります。AIが生成したコンテンツはますますリアルになり、真偽を見分けることが難しくなるでしょう。これにより、社会の混乱や不信感が増大する可能性があります。\n'
                        '\n'
                        'さらに、生成AIの進化は倫理的な問題も引き起こします。AIが人間のように振る舞うことで、責任の所在が曖昧になり、誤った判断や偏見が助長されるリスクがあります。技術の進化が必ずしも人類にとって良い方向に進むとは限らないのです。'}


In [20]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的AIです。二つの意見をまとめてください。"),
        ("human", "楽観的な意見: {optimistic_opinion}\n客観的意見: {pessimistic_opinion}"),
    ]
)

In [22]:
synthesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化に関する意見は、楽観的な見方と懸念を抱く見方の両方が存在します。楽観的な意見では、生成AIの進歩が私たちの生活をより便利で豊かにし、クリエイティブなプロジェクトの支援や日常業務の効率化、新しいアイデアの発掘など、多くの分野での応用が期待されています。特に教育や医療、環境保護といった重要な分野での影響が強調され、個々の学習スタイルに合わせた教材の提供や、医療診断の精度向上、新薬の開発促進などが可能になるとされています。

一方で、客観的な意見では、生成AIの進化に伴う懸念が指摘されています。AIの進化により人間の仕事が奪われる可能性があり、経済的不安が増大することが懸念されています。また、AIが生成するコンテンツの質が向上する一方で、偽情報やプロパガンダの拡散が容易になり、社会の分断が深まる恐れがあります。さらに、AI技術の悪用によるプライバシーの侵害やセキュリティの脆弱性の増加も問題視されています。

このように、生成AIの進化は多くの可能性を秘めている一方で、社会的、経済的な課題も伴うため、バランスの取れたアプローチが求められています。


Similar to how a single object is automatically converted into a Runnable, when we use a dict in a chain, it’s automatically converted into a `RunnableParallel`.

In [23]:
synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化に関する意見は、楽観的な見方と客観的な懸念の両方が存在します。楽観的な意見では、生成AIの進歩が私たちの生活を便利で豊かにし、クリエイティブなプロジェクトの支援や日常業務の効率化、教育、医療、エンターテインメントなど多くの分野での革新をもたらすと期待されています。また、異なる文化や言語の壁を越えて人々をつなげ、グローバルなコミュニケーションを促進する可能性も指摘されています。

一方で、客観的な意見では、生成AIの進化が必ずしも人類にとって良い方向に進むとは限らないとされています。AIが人間の仕事を奪う可能性があり、失業率の増加が懸念されています。また、AIが生成するコンテンツの質が向上する一方で、フェイクニュースや偽情報の拡散が巧妙になり、社会の混乱を招く恐れがあります。さらに、プライバシーの侵害や倫理的な問題も増加し、技術の進化が人間の生活を脅かす要因になる可能性も指摘されています。

このように、生成AIの進化には多くの可能性と同時に課題が存在し、これらをバランスよく考慮することが重要です。


## 5.3 itemgetter

`itemgetter` is part of the Python standard library and is often used with RunnableLambda. It provides an easy way to retrieve the value associated with a given key.

In [5]:
from operator import itemgetter

topic_getter = itemgetter("topic")
topic = topic_getter({"topic": "生成AIの進化について"})
print(topic)

生成AIの進化について


In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4o", temperature=0)
output_parser = StrOutputParser()

In [9]:
from langchain_core.prompts import ChatPromptTemplate

optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見をください。"),
        ("human", "{topic}"),
    ]
)
optimistic_chain = optimistic_prompt | model | output_parser

pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください。"),
        ("human", "{topic}"),
    ]
)
pessimistic_chain = pessimistic_prompt | model | output_parser

In [10]:
from operator import itemgetter

synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは客観的AIです。{topic}について二つの意見をまとめてください。",
        ),
        (
            "human",
            "楽観的意見: {optimistic_opinion}\n悲観的な意見: {pessimistic_opinion}",
        ),
    ]
)

synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
        "topic": itemgetter("topic"),   # `RunnableLambda(itemgetter)` is automatically executed
    }
    # Value obtained from `topic` will be fed into synthesize_prompt.
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化に関する意見は、楽観的な見方と悲観的な見方の両方が存在します。

楽観的な意見では、生成AIの進化は非常にエキサイティングであり、私たちの生活をより便利で豊かにする可能性を秘めているとされています。この技術の進歩は、クリエイティブなプロジェクトの支援や日常業務の効率化、新しいアイデアの発掘など、多岐にわたる分野での応用が期待されています。さらに、教育、医療、エンターテインメントなどの分野で革新をもたらし、多くの人々に新しい機会を提供し、より良い未来を築く手助けとなると考えられています。技術の進化によって、私たちの想像を超える素晴らしい可能性が広がることに期待が寄せられています。

一方、悲観的な意見では、生成AIの進化には多くの懸念が伴うとされています。AIの進化により、人間の仕事が奪われる可能性が高まり、多くの職業が自動化されることで職を失う人々が増えるかもしれません。また、AIが生成するコンテンツの質が向上する一方で、偽情報やプロパガンダの拡散が容易になり、社会に混乱をもたらすリスクも指摘されています。さらに、AIの進化に伴い、倫理的な問題やプライバシーの侵害といった新たな課題が浮上し、これらに対処するための法整備や規制が追いつかない可能性も懸念されています。全体として、生成AIの進化は多くの問題を引き起こす可能性があり、楽観視するのは難しいという見方もあります。


## 5.4 RunnablePassthrough

When intermediate chain values need to be displayed in the UI, `RunnablePassthrough.assign()` can be helpful.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。
                                          
文脈: """
{context}
"""
                                          
質問: {question}
''')

model = ChatOpenAI(model="gpt-4o", temperature=0)

In [4]:
from langchain_community.retrievers import TavilySearchAPIRetriever

# The parameter `k` specifies the number of search results to return.
retriever = TavilySearchAPIRetriever(k=3)

In [7]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}   # This part is automatically converted to `RunnableParallel`
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("東京の今日の天気は？")
print(output)

今日の東京の天気は「曇のち雨」です。昼頃から所々で雨雲が湧き、夜には広く雨が降る予報です。


Using `RunnablePassthrough.assign()` also outputs the retrieved content.

In [8]:
import pprint

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("東京の今日の天気は？")

# `pprint` automatically provides appropriate indentation, 
# line breaks, and alignment to make nested structures clear and easy to read.
pprint.pprint(output)

{'answer': '東京の今日の天気は「曇のち雨」です。昼頃から所々で雨雲が湧き、夜には広く雨になる予報です。',
 'context': [Document(metadata={'title': '東京都の天気 - 日本気象協会 tenki.jp', 'source': 'https://tenki.jp/forecast/3/16/', 'score': 0.7803451, 'images': []}, page_content='# tenki.jp 雨雲レーダー) 天気図 PM2.5分布予測 地震情報 日直予報士 熱中症情報 東京都の天気 ### 05月29日(木) 東京都の天気 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 晴時々曇 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 最新の天気履歴 渋谷区 曇のち雨 豊島区 曇のち雨 江東区 曇のち雨 港区 曇のち雨 大田区 曇のち雨 府中市 曇のち雨 奥多摩町 曇のち雨 関東・甲信 ### 気象予報士のポイント解説(日直予報士) newマーク 関東甲信\u3000今日29日は天気下り坂\u3000昼頃から所々で雨雲が湧く\u3000夜は広く雨に 関東甲信\u3000今日29日は天気下り坂\u3000昼頃から所々で雨雲が湧く\u3000夜は広く雨に newマーク 今日29日は気圧が低下\u3000沖縄や九州から近畿は影響度「大」の所も\u3000頭痛など注意 今日29日は気圧が低下\u3000沖縄や九州から近畿は影響度「大」の所も\u3000頭痛など注意 今日29日\u3000九州から関東は次第に雨\u3000東北と北海道は晴れて気温上昇\u3000真夏日も 今日29日\u3000九州から関東は次第に雨\u3000東北と北海道は晴れて気温上昇\u3000真夏日も ### 東京都各地の天気 #### 東京23区 #### 多摩 #### 伊豆諸島北部(大島) #### 伊豆諸島南部(八丈島) #### 小笠原諸島(父島) ### おすすめ記事 LINEの友達追加 ### 天気ガイド 雨雲 ### 注目の情報 アプリに便利なサブスクプラン開始 「tenki.jpライト」なら現在地の雨雲接近通知が受け取れる！ 新サービス「気圧予報」 気圧変化を確認して、頭痛やめまい、倦怠感

We can achieve the same effect as follows.

In [9]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("東京の今日の天気は？")
pprint.pprint(output)

{'answer': '東京の今日の天気は「曇のち雨」です。昼頃から所々で雨雲が湧き、夜には広く雨になる予報です。',
 'context': [Document(metadata={'title': '東京都の天気 - 日本気象協会 tenki.jp', 'source': 'https://tenki.jp/forecast/3/16/', 'score': 0.7803451, 'images': []}, page_content='# tenki.jp 雨雲レーダー) 天気図 PM2.5分布予測 地震情報 日直予報士 熱中症情報 東京都の天気 ### 05月29日(木) 東京都の天気 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 晴時々曇 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 最新の天気履歴 渋谷区 曇のち雨 豊島区 曇のち雨 江東区 曇のち雨 港区 曇のち雨 大田区 曇のち雨 府中市 曇のち雨 奥多摩町 曇のち雨 関東・甲信 ### 気象予報士のポイント解説(日直予報士) newマーク 関東甲信\u3000今日29日は天気下り坂\u3000昼頃から所々で雨雲が湧く\u3000夜は広く雨に 関東甲信\u3000今日29日は天気下り坂\u3000昼頃から所々で雨雲が湧く\u3000夜は広く雨に newマーク 今日29日は気圧が低下\u3000沖縄や九州から近畿は影響度「大」の所も\u3000頭痛など注意 今日29日は気圧が低下\u3000沖縄や九州から近畿は影響度「大」の所も\u3000頭痛など注意 今日29日\u3000九州から関東は次第に雨\u3000東北と北海道は晴れて気温上昇\u3000真夏日も 今日29日\u3000九州から関東は次第に雨\u3000東北と北海道は晴れて気温上昇\u3000真夏日も ### 東京都各地の天気 #### 東京23区 #### 多摩 #### 伊豆諸島北部(大島) #### 伊豆諸島南部(八丈島) #### 小笠原諸島(父島) ### おすすめ記事 LINEの友達追加 ### 天気ガイド 雨雲 ### 注目の情報 アプリに便利なサブスクプラン開始 「tenki.jpライト」なら現在地の雨雲接近通知が受け取れる！ 新サービス「気圧予報」 気圧変化を確認して、頭痛やめまい、倦怠感

We can use `RunnablePassthrough.assign().pick()` to determine the final output component.

In [11]:
chain = (
    RunnableParallel(
        {
            "question": RunnablePassthrough(),
            "context": retriever,
        }
    )
    .assign(answer=prompt | model | StrOutputParser())
    .pick(["context", "answer"])
)

output = chain.invoke("東京の今日の天気は？")
pprint.pprint(output)

{'answer': '東京の今日の天気は「曇のち雨」です。昼頃から所々で雨雲が湧き、夜には広く雨になる予報です。',
 'context': [Document(metadata={'title': '東京都の天気 - 日本気象協会 tenki.jp', 'source': 'https://tenki.jp/forecast/3/16/', 'score': 0.7803451, 'images': []}, page_content='# tenki.jp 雨雲レーダー) 天気図 PM2.5分布予測 地震情報 日直予報士 熱中症情報 東京都の天気 ### 05月29日(木) 東京都の天気 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 晴時々曇 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 最新の天気履歴 渋谷区 曇のち雨 豊島区 曇のち雨 江東区 曇のち雨 港区 曇のち雨 大田区 曇のち雨 府中市 曇のち雨 奥多摩町 曇のち雨 関東・甲信 ### 気象予報士のポイント解説(日直予報士) newマーク 関東甲信\u3000今日29日は天気下り坂\u3000昼頃から所々で雨雲が湧く\u3000夜は広く雨に 関東甲信\u3000今日29日は天気下り坂\u3000昼頃から所々で雨雲が湧く\u3000夜は広く雨に newマーク 今日29日は気圧が低下\u3000沖縄や九州から近畿は影響度「大」の所も\u3000頭痛など注意 今日29日は気圧が低下\u3000沖縄や九州から近畿は影響度「大」の所も\u3000頭痛など注意 今日29日\u3000九州から関東は次第に雨\u3000東北と北海道は晴れて気温上昇\u3000真夏日も 今日29日\u3000九州から関東は次第に雨\u3000東北と北海道は晴れて気温上昇\u3000真夏日も ### 東京都各地の天気 #### 東京23区 #### 多摩 #### 伊豆諸島北部(大島) #### 伊豆諸島南部(八丈島) #### 小笠原諸島(父島) ### おすすめ記事 LINEの友達追加 ### 天気ガイド 雨雲 ### 注目の情報 アプリに便利なサブスクプラン開始 「tenki.jpライト」なら現在地の雨雲接近通知が受け取れる！ 新サービス「気圧予報」 気圧変化を確認して、頭痛やめまい、倦怠感

### chain.astream_events()

Using `chain.astream_events()` will print the events for each stage of the chain.


In [12]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

async for event in chain.astream_events("東京の今日の天気は？", version="v2"):
    print(event, flush=True)

{'event': 'on_chain_start', 'data': {'input': '東京の今日の天気は？'}, 'name': 'RunnableSequence', 'tags': [], 'run_id': 'fda32799-d750-43e9-b8eb-ddc3aa379043', 'metadata': {}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {}, 'name': 'RunnableParallel<context,question>', 'tags': ['seq:step:1'], 'run_id': '059bd87c-1070-4592-b12e-affc3fb186d5', 'metadata': {}, 'parent_ids': ['fda32799-d750-43e9-b8eb-ddc3aa379043']}
{'event': 'on_retriever_start', 'data': {'input': {'query': '東京の今日の天気は？'}}, 'name': 'TavilySearchAPIRetriever', 'tags': ['map:key:context'], 'run_id': 'cde42286-a560-475a-ba77-037c5c72a361', 'metadata': {'ls_retriever_name': 'tavilysearchapi'}, 'parent_ids': ['fda32799-d750-43e9-b8eb-ddc3aa379043', '059bd87c-1070-4592-b12e-affc3fb186d5']}
{'event': 'on_chain_start', 'data': {}, 'name': 'RunnablePassthrough', 'tags': ['map:key:question'], 'run_id': '63d05043-6951-4829-af0a-3ac8d4e198a2', 'metadata': {}, 'parent_ids': ['fda32799-d750-43e9-b8eb-ddc3aa379043', '059bd87c-1070-4592-

In [14]:
async for event in chain.astream_events("東京の今日の天気は？", version="v2"):
    event_kind = event["event"]

    if event_kind == "on_retriever_end":
        print("==== Retrieve Results ===")
        documents = event["data"]["output"]
        for document in documents:
            print(document)
    
    elif event_kind == "on_parser_start":
        print("=== Final Output ===")
    
    elif event_kind == "on_parser_stream":
        chunk = event["data"]["chunk"]
        print(chunk, end="", flush=True)

==== Retrieve Results ===
page_content='# tenki.jp 雨雲レーダー) 天気図 PM2.5分布予測 地震情報 日直予報士 熱中症情報 東京都の天気 ### 05月29日(木) 東京都の天気 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 晴時々曇 曇のち雨 曇のち雨 曇のち雨 曇のち雨 曇のち雨 最新の天気履歴 渋谷区 曇のち雨 豊島区 曇のち雨 江東区 曇のち雨 港区 曇のち雨 大田区 曇のち雨 府中市 曇のち雨 奥多摩町 曇のち雨 関東・甲信 ### 気象予報士のポイント解説(日直予報士) newマーク 関東甲信　今日29日は天気下り坂　昼頃から所々で雨雲が湧く　夜は広く雨に 関東甲信　今日29日は天気下り坂　昼頃から所々で雨雲が湧く　夜は広く雨に newマーク 今日29日は気圧が低下　沖縄や九州から近畿は影響度「大」の所も　頭痛など注意 今日29日は気圧が低下　沖縄や九州から近畿は影響度「大」の所も　頭痛など注意 今日29日　九州から関東は次第に雨　東北と北海道は晴れて気温上昇　真夏日も 今日29日　九州から関東は次第に雨　東北と北海道は晴れて気温上昇　真夏日も ### 東京都各地の天気 #### 東京23区 #### 多摩 #### 伊豆諸島北部(大島) #### 伊豆諸島南部(八丈島) #### 小笠原諸島(父島) ### おすすめ記事 LINEの友達追加 ### 天気ガイド 雨雲 ### 注目の情報 アプリに便利なサブスクプラン開始 「tenki.jpライト」なら現在地の雨雲接近通知が受け取れる！ 新サービス「気圧予報」 気圧変化を確認して、頭痛やめまい、倦怠感といった症状に備えましょう。 X（旧Twitter） tenki.jpの公式X（旧Twitter） 最新の気象・防災情報や、生活に役立つ情報を毎日リアルタイムに配信中！ 天気予報 観測 防災情報 天気図 指数情報 レジャー天気 季節特集 天気ニュース X(旧：Twitter) Youtube Facebook Instagram LINEの友達追加 tenki.jp tenki.jp tenki.jp 登山天気 tenki.jp 登山天気 全国のコンテンツ tenki.jpトップ 天気予報 観測 防災情報 天気図' me

## 5.5 Chat History